In [1]:
import pyodbc
import pandas as pd
from sqlalchemy import create_engine
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from office365.runtime.auth.user_credential import UserCredential
from io import BytesIO
import pyodbc
import pandas as pd
import re
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
import pandas as pd
from io import StringIO
from office365.runtime.auth.user_credential import UserCredential
import smtplib
from email.message import EmailMessage
import os
from datetime import datetime, timedelta


In [2]:
server = 'server'
database = 'database'
username = 'User'
password = 'password'

conn = pyodbc.connect(f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}')

query = """

SELECT * from where order by group by;



"""
df = pd.read_sql(query, conn)

conn.close()

C:\Users\DarshanaPrasad\AppData\Local\Temp\ipykernel_17072\3585699339.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


DatabaseError: Execution failed on sql '

SELECT 
    COALESCE(A.PRODUCT_CODE, B.PRODUCT_CODE, C.PRODUCT_CODE ,D.[ITEM CODE]) AS PRODUCT_CODE,
    A.AVAILABLE_STOCK AS KOTTE_STOCK,
    B.AVAILABLE_STOCK AS COLOMBO_STOCK,
    C.AVAILABLE_STOCK AS ATTIDIYA_STOCK,
    MAX(CASE WHEN D.BC_LOCATION = 'PWH' THEN D.ITEM_SIH END) AS PWH_STOCK,
    MAX(CASE WHEN D.BC_LOCATION = 'MWH' THEN D.ITEM_SIH END) AS MWH_STOCK
FROM [myPOS_DB].[dbo].[API_STOCK_KOTTE_OUTLET] A
FULL OUTER JOIN [myPOS_DB].[dbo].[API_STOCK_COLOMBO_OUTLET] B 
    ON A.PRODUCT_CODE = B.PRODUCT_CODE
FULL OUTER JOIN [myPOS_DB].[dbo].[API_STOCK_ATTIDIYA_OUTLET] C 
    ON COALESCE(A.PRODUCT_CODE, B.PRODUCT_CODE) = C.PRODUCT_CODE
LEFT JOIN [myPOS_DB].[dbo].[AAA_ITEM_LISTING] D
    ON COALESCE(A.PRODUCT_CODE, B.PRODUCT_CODE, C.PRODUCT_CODE) = D.[ITEM CODE]
WHERE COALESCE(A.PRODUCT_CODE, B.PRODUCT_CODE, C.PRODUCT_CODE, D.[ITEM CODE]) LIKE '0%'
GROUP BY COALESCE(A.PRODUCT_CODE, B.PRODUCT_CODE, C.PRODUCT_CODE ,D.[ITEM CODE]),
         A.

In [ ]:
# Remove leading zeros from PRODUCT_CODE in df_sql
df['PRODUCT_CODE'] = df['PRODUCT_CODE'].astype(str).str.lstrip('0')
df.head()

In [ ]:
pip install xlsxwriter

In [ ]:
from office365.sharepoint.client_context import ClientContext
from office365.runtime.auth.user_credential import UserCredential
import pandas as pd
from io import BytesIO

# SharePoint site and authentication details
site_url = "https://lk.sharepoint.com/sites"
username = "emailsharepoint"
password = "emailpassword"

# SharePoint file URL
file_url = "/sites/Stock/Shared%20Documents/Transfer-Outs%20Request/dont_touch/sku.xlsx"
#https://lk.sharepoint.com/sites/Stock/Shared%20Documents/Transfer-Outs%20Request/dont_touch/sku.xlsx
# Authenticate SharePoint context
ctx = ClientContext(site_url).with_credentials(UserCredential(username, password))

# Get the file from SharePoint
file = ctx.web.get_file_by_server_relative_url(file_url)
file_object = BytesIO()
file.download(file_object).execute_query()
file_object.seek(0)  # Reset buffer position

# Read all sheets from the SharePoint file
sku_sheets = pd.read_excel(file_object, sheet_name=None)

# Choose the correct sheet
sheet_name = "sku"  # Update if needed
if sheet_name in sku_sheets:
    sku_sheet_df = sku_sheets[sheet_name]
    
    # Ensure both columns have the same data type before merging
    sku_sheet_df['Items Code'] = sku_sheet_df['Items Code'].astype(str)
    df['PRODUCT_CODE'] = df['PRODUCT_CODE'].astype(str)  # Ensure 'df' is defined
    
    # Perform the merge
    merge_df = pd.merge(df, sku_sheet_df, left_on='PRODUCT_CODE', right_on='Items Code', how='right')
    
    # Display merged DataFrame
    print(merge_df.head())  # Show first few rows
    
    # Save merge_df as a new Excel file
    excel_buffer = BytesIO()
    with pd.ExcelWriter(excel_buffer, engine='xlsxwriter') as writer:
        merge_df.to_excel(writer, index=False, sheet_name='Merged_Data')  # Save merge_df to a new sheet
        
    excel_buffer.seek(0)  # Reset buffer position
    
    # Upload the Excel file back to SharePoint
    target_folder_url = "/sites/Stock/Shared Documents/Transfer-Outs Request/dont_touch"
    target_file_name = "merged_data.xlsx"  # New file name for the merged data
    target_folder = ctx.web.get_folder_by_server_relative_url(target_folder_url)
    
    # Check if the file already exists and delete it
    try:
        existing_file = target_folder.files.get_by_url(target_file_name)
        existing_file.delete_object().execute_query()
        print(f"Existing file '{target_file_name}' deleted.")
    except Exception as e:
        print(f"No existing file to delete: {e}")
    
    # Upload the new file
    target_file = target_folder.upload_file(target_file_name, excel_buffer).execute_query()
    
    # Print the server-relative URL of the uploaded file
    print(f"File '{target_file.serverRelativeUrl}' has been uploaded.")
else:
    print(f"Error: Sheet '{sheet_name}' not found in SKU file.")
